In [1]:
import json
import os
from backend.constants import *

file = open('../../arena/evaluation_sanitized-mbpp.json', 'r')
tasks = json.load(file)  


code = ""
for task in tasks:
    code += task["code"] + "\n\n"

In [2]:
import os
import uuid

# Define the folder and file structure
version_folder = "lasso-llm-0.0.1"
subfolder = "lasso_llm"
file_name = f"{uuid.uuid4()}.py"

# Full path creation
full_path = os.path.join(INSTALLED, version_folder, subfolder)
os.makedirs(full_path, exist_ok=True)

# Full path to the file
file_path = os.path.join(full_path, file_name)

# Write the content of the `code` variable into the Python file
with open(file_path, 'w') as file:
    file.write(code)
    
# Create an __init__.py file to make the subfolder a package, including the version
init_file_path = os.path.join(full_path, '__init__.py')
with open(init_file_path, 'w') as init_file:
    init_file.write(f'"""lasso-llm package."""\n')
    init_file.write(f'__version__ = "0.0.1"\n')

print(f"File created at: {file_path}")

File created at: /Users/maxlautenbach/Documents/GitHub/lasso-python/backend/crawl/installed/lasso-llm-0.0.1/lasso_llm/239a4f90-d701-4922-a376-c3c17847804b.py


In [3]:
from backend.crawl import import_helper

imp_helper = import_helper.ImportHelper()
imp_helper.pre_load_package("lasso-llm", "0.0.1")

In [4]:
import lasso_llm

In [5]:
from backend.crawl.splitting import get_module_index

index = get_module_index("lasso_llm","lasso_llm","0.0.1")

lasso_llm indexed


In [6]:
from backend.crawl.nexus import Package, Nexus

pkg = Package("lasso-llm", "0.0.1")
pkg.compress()
nexus = Nexus()
nexus.upload(pkg)

Nexus is up and running!
Failed to upload /Users/maxlautenbach/Documents/GitHub/lasso-python/backend/crawl/installed/lasso-llm-0.0.1.tar.gz. HTTP Status Code: 404


False